In [4]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [5]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [6]:
#train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [8]:
only_band1 = ['band1_'+str(i) for i in range(75*75)]
only_band2 = ['band2_'+str(i) for i in range(75*75)]
both_bands = only_band1+only_band2
df = pd.concat([pd.DataFrame(test['band_1'].values.tolist(),columns = only_band1),
                pd.DataFrame(test['band_2'].values.tolist(),columns = only_band2)],axis = 1)

In [12]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [14]:
test_data = collect_test()

In [14]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [5]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [25]:

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Cross Validation Folds

In [6]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [7]:
def basic_cbamd_layer(X_input,filter_num =64,kernel_size = (3,3),conv_padding = 'valid',pool_size =(2,2),pool_strides = (2,2),pool_padding = 'valid',dropout_rate = 0.2):
    X = Conv2D(filter_num, kernel_size, padding = conv_padding)(X_input)
    #X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size,strides = pool_strides,padding = pool_padding)(X)
    X = Dropout(rate = dropout_rate)(X)
    return X

def dense_layer(X_input, units = 1,activation = 'relu',dropout_rate = 0.2):
    X = Dense(units = units,activation = activation)(X_input)
    X = Dropout(rate = dropout_rate)(X)
    return X
#kernel_regularizer=regularizers.l2(0.01)
def cnnmodel(input_shape,lr = 0.0001):
    X_input = Input(input_shape)
    #First composite layer
    X = basic_cbamd_layer(X_input,
                           filter_num =64,
                           kernel_size = (5,5),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Second composite layer
    X = basic_cbamd_layer(X,
                           filter_num =128,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Third composite layer
    X = basic_cbamd_layer(X,
                           filter_num =128,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(2,2),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Forth composite layer
    X = basic_cbamd_layer(X,
                           filter_num =64,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Flatten layer
    X = Flatten()(X)
    #First dense layer
    X = dense_layer(X, units = 256,activation = 'relu',dropout_rate = 0.2)
    #Second dense layer
    X = dense_layer(X, units = 128,activation = 'relu',dropout_rate = 0.2)
    #Decision layer
    X = dense_layer(X, units = 1,activation = 'sigmoid',dropout_rate = 0)
    
    model = Model(inputs=X_input,outputs=X)
    optimizer=Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7):
    #es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave]

#callbacks = get_callbacks(filepath=file_path, patience=3)

In [8]:

def fitmodel(model,X_train,y_train,X_valid,y_valid,augment = False,epochs = 50,batch_size = 32,filepath = ".model_weights.hdf5"):
    if augment == False:
        result = model.fit(X_train, y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(X_valid, y_valid),
                          callbacks=get_callbacks(filepath))
    else:
        result = model.fit_generator(data_augmentation(X_train,y_train,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(X_valid, y_valid),
                          callbacks=get_callbacks(filepath))
    return result

In [34]:
for i,layer in enumerate(Cnnmodel.layers):
    print(i,layer.name)

0 input_13
1 conv2d_49
2 activation_49
3 max_pooling2d_49
4 dropout_85
5 conv2d_50
6 activation_50
7 max_pooling2d_50
8 dropout_86
9 conv2d_51
10 activation_51
11 max_pooling2d_51
12 dropout_87
13 conv2d_52
14 activation_52
15 max_pooling2d_52
16 dropout_88
17 flatten_13
18 dense_37
19 dropout_89
20 dense_38
21 dropout_90
22 dense_39
23 dropout_91


In [36]:
Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
Cnnmodel.load_weights('model save/Model 1-Advanced CNN with DA and CV5/2018.1.1_local0.2069OF.hdf5')
for layer in Cnnmodel.layers[:5]:
    layer.trainable = False
Cnnmodel.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                  metrics=['accuracy'])
Cnnmodel.summary()
result_Cnn = fitmodel(Cnnmodel,X_train_cv,y_train_cv,X_valid,y_valid,augment = True,epochs = 200,batch_size = 32,filepath = 'model save/Model 1-Advanced CNN with DA and CV5/2018.1.1.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_57 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_99 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_58 (Activation)   (None, 33, 33, 128)       0         
__________

Epoch 10/200
41/40 [==============================] - 4s 95ms/step - loss: 0.2142 - acc: 0.9131 - val_loss: 0.2126 - val_acc: 0.9065
Epoch 11/200
41/40 [==============================] - 4s 95ms/step - loss: 0.2458 - acc: 0.9036 - val_loss: 0.2135 - val_acc: 0.9128
Epoch 12/200
41/40 [==============================] - 4s 100ms/step - loss: 0.2163 - acc: 0.9039 - val_loss: 0.2163 - val_acc: 0.9034
Epoch 13/200
41/40 [==============================] - 4s 100ms/step - loss: 0.2187 - acc: 0.9051 - val_loss: 0.2076 - val_acc: 0.9283
Epoch 14/200
41/40 [==============================] - 4s 92ms/step - loss: 0.2443 - acc: 0.8827 - val_loss: 0.2226 - val_acc: 0.8941
Epoch 15/200
41/40 [==============================] - 4s 96ms/step - loss: 0.1971 - acc: 0.9131 - val_loss: 0.2086 - val_acc: 0.9128
Epoch 16/200
41/40 [==============================] - 4s 98ms/step - loss: 0.2126 - acc: 0.9123 - val_loss: 0.2188 - val_acc: 0.9065
Epoch 17/200
41/40 [==============================] - 4s 94ms/step 

KeyboardInterrupt: 

In [44]:
Cnnmodel.load_weights('model save/Model 1-Advanced CNN with DA and CV5/2018.1.1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

321/321 [==============================] - 0s 1ms/step


[0.18129999453858855, 0.94392523364485981]

In [149]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [137]:
save_history(result_Cnn)

NameError: name 'result_Cnn' is not defined

### Prediction

In [12]:
def Model6_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 2).split(X_train, target_train))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        result_Cnn = fitmodel(Cnnmodel,X_train_cv,y_train_cv,X_valid,y_valid,
                              augment = False,epochs = 100,batch_size = 32,
                              filepath = file_path)
        print('Satge 1,done!')
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)
        for layer in Cnnmodel.layers[:5]:
            layer.trainable = False
        Cnnmodel.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                  metrics=['accuracy'])
        Cnnmodel.summary()
        result_Cnn = fitmodel(Cnnmodel,X_train_cv,y_train_cv,X_valid,y_valid,
                              augment = True,epochs = 150,batch_size = 32,
                              filepath = file_path)
        print('Satge 2,done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [15]:
Model6_CV(X_train,K = 5)


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_1 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_2 (Activation)    (None, 33, 33, 128)

KeyboardInterrupt: 

### Submission

In [18]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 2).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [19]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_9 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_10 (Activation)   (None, 33, 33, 128)

_________________________________________________________________
conv2d_18 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_18 (Activation)   (None, 33, 33, 128)       0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_19 (Activation)   (None, 14, 14, 128)       0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_31

dropout_45 (Dropout)         (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 5, 5, 64)          73792     
_________________________________________________________________
activation_28 (Activation)   (None, 5, 5, 64)          0         
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_47 (Dropout)         (None, 256)               0         
__________

### Submission(min max median stacking)

In [35]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

In [36]:
test_df = test_scores()


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_29 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_50 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_30 (Activation)   (None, 33, 33, 128)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':



===================FOLD= 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_33 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_57 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_34 (Activation)   (None, 33, 33, 128)


===================FOLD= 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_45 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_78 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_46 (Activation)   (None, 33, 33, 128)

In [45]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [46]:
sub = get_sub_mmm(test_df)